In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split # División del dataset
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D

In [3]:
df_Retail = pd.read_csv('../data/new_retail_data.csv')

In [4]:
df_Retail.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [5]:
df_Retail.shape

(302010, 30)

In [6]:
df_Retail.dtypes

Transaction_ID      float64
Customer_ID         float64
Name                 object
Email                object
Phone               float64
Address              object
City                 object
State                object
Zipcode             float64
Country              object
Age                 float64
Gender               object
Income               object
Customer_Segment     object
Date                 object
Year                float64
Month                object
Time                 object
Total_Purchases     float64
Amount              float64
Total_Amount        float64
Product_Category     object
Product_Brand        object
Product_Type         object
Feedback             object
Shipping_Method      object
Payment_Method       object
Order_Status         object
Ratings             float64
products             object
dtype: object

##Analizar si hay missing values o valores nulos

In [8]:
df_Retail.isna().sum().sum() # number of missing cells
round(df_Retail.isna().sum().sum() / df_Retail.size * 100, 1) # percentage of missing cells

0.1

EL 10% tiene datos faltantes

In [9]:
df_Retail.isna().sum()

Transaction_ID      333
Customer_ID         308
Name                382
Email               347
Phone               362
Address             315
City                248
State               281
Zipcode             340
Country             271
Age                 173
Gender              317
Income              290
Customer_Segment    215
Date                359
Year                350
Month               273
Time                350
Total_Purchases     361
Amount              357
Total_Amount        350
Product_Category    283
Product_Brand       281
Product_Type          0
Feedback            184
Shipping_Method     337
Payment_Method      297
Order_Status        235
Ratings             184
products              0
dtype: int64

Por mayor comodidad, transformo todas las variables a category menos las numericas

In [10]:
columns_not_to_convert = ['Age', 'Zipcode','Total_Purchases', 'Amount', 'Total_Amount', 'Ratings', 'Customer_ID', 'Transaction_ID']
all_columns = set(df_Retail.columns)

columns_not_to_convert_set = set(columns_not_to_convert)
columns_to_skip = list(all_columns - columns_not_to_convert_set)

# Convertir las columnas restantes a tipo 'category'
for col in columns_to_skip:
    df_Retail[col] = df_Retail[col].astype('category')

In [13]:
df_Retail['Age'].unique()

array([21, 19, 48, 56, 22, 58, 29, 46, 25, 64, 31, 53, 32, 43, 69, 49, 61,
       41, 38, 59, 20, 67, 50, 26, 66, 24, 54, 28, 34, 65, 40, 68, 36, 57,
       27, 35, 70, 37, 30, 39, 47, 18, 60, 33, 42, 44, 51, 63, 55, 23, 62,
       52, 45])

La edad esta en float, la transformo a entero

In [11]:
df_Retail = df_Retail.dropna(subset=['Age'])
df_Retail = df_Retail.dropna(subset=['Zipcode'])
df_Retail = df_Retail.dropna(subset=['Year'])

In [12]:
df_Retail['Age'] = df_Retail['Age'].astype('int')
df_Retail['Year'] = df_Retail['Year'].astype('int')
df_Retail['Zipcode'] = df_Retail['Zipcode'].astype('int')

Quiero borrar todos los valores nulos o nan pero quiero estudiar primero si me conviene borrar las filas o una columna entera, porque si los datos faltantes es tan en su mayoria en una columna puede ser mejor deshacerme de esa columna en lugar de borrar registros.
A continuacion estudio cual es la variable que tiene la mayor cantidad de nulos o valores nan

In [14]:
# Calcular el número de valores NaN por columna
nan_counts = df_Retail.isna().sum()

# Encontrar la columna con el mayor número de NaN
column_with_most_nan = nan_counts.idxmax()
max_nan_count = nan_counts.max()

# Mostrar resultados
print("La columna con la mayor cantidad de NaN es:", column_with_most_nan)
print("Número de NaN en esta columna:", max_nan_count)

La columna con la mayor cantidad de NaN es: Name
Número de NaN en esta columna: 382


Siendo que Name es la columna con la mayor cantidad de nulos y que esta no tiene valor estadistico para mi estudio, la elimino

In [15]:
df_Retail = df_Retail.drop(columns=['Name'])

In [16]:
df_Retail['Total_Amount'] = df_Retail['Total_Amount'].round(2)
df_Retail['Amount'] = df_Retail['Amount'].round(2)
df_Retail['Total_Purchases'] = df_Retail['Total_Purchases'].round(2)

In [17]:
# Quiero eliminar las filas que contengan NAN, pero no quiero quedarme con pocos datos por lo que cuento primero cuantas filas se borrarian

total_rows_before = df_Retail.shape[0]

# Eliminar todas las filas que contienen al menos un NaN en alguna columna
df_clean = df_Retail.dropna()

# Contar las filas después de eliminar NaN
total_rows_after = df_clean.shape[0]

# Calcular cuántas filas se eliminaron
rows_deleted = total_rows_before - total_rows_after

# Mostrar resultados
print("Número total de filas antes de eliminar NaN:", total_rows_before)
print("Número total de filas después de eliminar NaN:", total_rows_after)
print("Número de filas eliminadas:", rows_deleted)

Número total de filas antes de eliminar NaN: 301147
Número total de filas después de eliminar NaN: 294280
Número de filas eliminadas: 6867


Estudio la consistencia de la variable edad

In [19]:
# Verificar si hay clientes con edad menor a 18
underage_clients = (df_clean['Age'] < 18).any()

# Mostrar resultado
if underage_clients:
    print("Hay clientes cuya edad es menor a 18.")
else:
    print("No hay clientes cuya edad sea menor a 18.")

No hay clientes cuya edad sea menor a 18.


In [20]:
df_clean = df_clean.drop(columns=['Phone'])

In [21]:
df_clean['Customer_ID'] = df_clean['Customer_ID'].astype('int')
df_clean['Transaction_ID'] = df_clean['Transaction_ID'].astype('int')

In [22]:
# Encontrar valores que se repiten en 'Transaction_ID'
duplicate_values = df_clean[df_clean.duplicated(subset=['Transaction_ID'], keep=False)]

# Guardar duplicate_values como DataFrame
df_duplicate_values = pd.DataFrame(duplicate_values)
df_duplicate_values.head()

,Transaction_ID,Customer_ID,Email,Address,City,State,Zipcode,Country,Age,Gender,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788,37249,Ebony39@gmail.com,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,21,Male,...,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
32,8973612,65701,Derrick51@gmail.com,57136 Martha Common Apt. 654,Portsmouth,England,11531,UK,66,Male,...,2412.75,Home Decor,IKEA,Decorations,Excellent,Same-Day,Debit Card,Processing,5.0,Curtains
42,3902930,77572,Amber67@gmail.com,02936 Roy Village,Portsmouth,England,98771,UK,68,Female,...,2004.99,Electronics,Apple,Laptop,Bad,Same-Day,Debit Card,Shipped,1.0,Microsoft Surface Laptop
72,9534452,20500,Kara43@gmail.com,51720 Rachael Spurs Suite 283,Portsmouth,England,94447,UK,61,Female,...,807.81,Clothing,Zara,Dress,Average,Standard,PayPal,Processing,2.0,Maxi dress
93,1874788,74649,Tina32@gmail.com,571 Samuel Wall Apt. 895,Portsmouth,England,24895,UK,18,Male,...,1445.58,Grocery,Nestle,Chocolate,Average,Express,Debit Card,Delivered,2.0,Chocolate-covered fruits


In [23]:
df_filtered = df_clean[df_clean['Transaction_ID'] == 8691788]
df_filtered.head()

,Transaction_ID,Customer_ID,Email,Address,City,State,Zipcode,Country,Age,Gender,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788,37249,Ebony39@gmail.com,3959 Amanda Burgs,Dortmund,Berlin,77985,Germany,21,Male,...,324.09,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
112551,8691788,57392,Krystal93@gmail.com,32467 Lopez Falls Suite 272,Ottawa,Ontario,88811,Canada,46,Male,...,294.76,Electronics,Sony,Television,Excellent,Standard,Cash,Processing,4.0,LED TV


Este resultado es inconsistente, muestra para una misma trasaccion distintos customer_id

In [24]:
df_clean_unique = df_clean.drop_duplicates(subset=['Transaction_ID'], keep=False)

In [26]:
# Contar las compras por año
compras_por_año = df_clean_unique['Year'].value_counts().sort_index()

# Mostrar el conteo de compras por año
print("Conteo de Compras por Año:")
print(compras_por_año)

Conteo de Compras por Año:
Year
2023    234312
2024     46247
Name: count, dtype: int64


Hay pocos datos para el 2024, por lo que me voy a centrar en el 2023

In [27]:
df_clean_unique = df_clean_unique.loc[df_clean_unique['Year'] != 2024]

In [28]:
df_clean_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234312 entries, 1 to 301262
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Transaction_ID    234312 non-null  int32   
 1   Customer_ID       234312 non-null  int32   
 2   Email             234312 non-null  category
 3   Address           234312 non-null  category
 4   City              234312 non-null  category
 5   State             234312 non-null  category
 6   Zipcode           234312 non-null  int32   
 7   Country           234312 non-null  category
 8   Age               234312 non-null  int32   
 9   Gender            234312 non-null  category
 10  Income            234312 non-null  category
 11  Customer_Segment  234312 non-null  category
 12  Date              234312 non-null  category
 13  Year              234312 non-null  int32   
 14  Month             234312 non-null  category
 15  Time              234312 non-null  category
 16  Total_P